# Homework 1: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of four are not allowed.
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
If a topic was not covered in detail during the lecture, you are asked to study it online on your own. 
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

For your convenience, , the code has a **DEBUG** mode that you may use in order to debug with toy data.  
It is recommended to solve the code in that mode (with efficiency in mind) and then run the code on all the data.
**Do not forget to file the HW with DEBUG == False**.

Download the "Lyrics" dataset from Moodle and put it in the same directory your script is.


5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
from pathlib import Path

In [2]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\netac\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\netac\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


# Debug
**you can change this cell**

In [3]:
DEBUG = True
DEBUG = False

"""
Recommended to start with a small number to get a feeling for the preprocessing with prints (N_ROWS_FOR_DEBUG = 2)
later increase this number for 5*10**3 in order to see that the code runs at reasonable speed, and change the CHUNK_SIZE accordinaly
When setting Debug == False, our code implements bow.fit() in 15-20 minutes according to the tqdm progress bar. Your solution is not supposed to be much further than that.
"""
N_ROWS_FOR_DEBUG = 5
CHUNCK_SIZE = 1 if DEBUG else 5*10**3

# Config

In [4]:
INPUT_FILE_PATH = Path("lyrics.csv")
BOW_PATH = Path("bow.csv")
N_ROWS = N_ROWS_FOR_DEBUG if DEBUG else None
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1 if DEBUG else 363*10**3//CHUNCK_SIZE + 1
COLS = [5]

## 1.1 Bag of words model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update self.trigram count (three consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [7]:
def preprocess_sentence(sentence : str) -> List[str]:
    output_sentence = []
    for word in word_tokenize(sentence):
        word_to_insert = ''.join([c for c in word.lower() if c in allowed_symbols])
        word_to_insert = stemmer.stem(word_to_insert)
        if word_to_insert not in stop_words and len(word_to_insert)>1: output_sentence.append(word_to_insert)       
    return output_sentence
    

def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

class TfIdf:
    def __init__(self):
        self.unigram_count =  Counter()
        self.bigram_count = Counter()
        self.trigram_count = Counter()
        self.document_term_frequency = Counter()
        self.word_document_frequency = {}
        self.inverted_index = {}
        self.doc_norms = {}
        self.n_docs = -1
        self.sentence_preprocesser = preprocess_sentence
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence :List[str],document_id:int) -> None:
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            self.unigram_count[word] += 1
            if i > 0: self.bigram_count[(sentence[i-1],sentence[i])] += 1
            if i > 1: self.trigram_count[(sentence[i-2],sentence[i-1],sentence[i])] += 1
                
            #Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}
            if word not in self.inverted_index: self.inverted_index[word] = {}
            if document_id not in self.inverted_index[word]: self.inverted_index[word][document_id] = 0
            self.inverted_index[word][document_id] += 1     
        
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck: #sentence is a song (string)
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
             
    def compute_word_document_frequency(self):
        for word in self.inverted_index.keys():
            #For each word count the number of docs it appears in. For example , for the word 'apple' -
            self.word_document_frequency[word] = len(self.inverted_index[word])
            
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        doc_count={}
        for word in self.inverted_index.keys():
            for doc in self.inverted_index[word]:
                if doc not in doc_count: doc_count[doc] = 0
                doc_count[doc] += self.inverted_index[word][doc]
                if doc not in self.doc_norms: self.doc_norms[doc] = 0   
                self.doc_norms[doc] += self.inverted_index[word][doc]**2
    
        for word in self.inverted_index.keys():
            for doc in self.inverted_index[word]:
                tf = self.inverted_index[word][doc]/doc_count[doc]#self.doc_norms[doc]
                df = self.word_document_frequency[word]
                idf = np.log10(self.n_docs/df)
                self.inverted_index[word][doc] = tf*idf

                
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)
                
tf_idf = TfIdf()
tf_idf.fit()

100%|██████████| 73/73 [26:01<00:00, 21.39s/it]


**You need to run the TfIdf model without the DEBUG mode until this stage**

## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

### YOUR SOLUTION HERE
1. What is the computational complexity of this model, as a function of the number of docs in the corpus?

Answer: The copmputational complexity of this model is O(|D||V|), where |D| in the number of documents and |V| is the maximum number of words in a document. Explanation:

A Starting from the fit function we have the first for loop, iterating through all sentesnces=documents ==> (O|D|).

   A.1 before calling a fuction we do some O(1) arithmetic operations
   
   A.2 Then for each iteration in the "fit" function "for" loop, we go first to the "preprocess_sentence" function wehre we  
   iterate through all the words in a sentence --> maximum O(|V|) ==> O(|D||V|)
   
   A.3. After the above fuction, for each document we go to the "update_counts_and_probabilities" func. In this function we 
   first find the documents dictionary for a specific word, which takes O(1) becasue python stores dictionaries as Hash tables. 
   and after fining the list of documents for a specific word, finding if a document is within a dictionary is also done using 
   Hash table ==> O(1). We also finish the For loop.
  Total Complexity for the For loop - O(|D||V|)
  
   
B. After the For loop we use save_bow() to save the BOW ==> O(|D||V|)

C. compute_word_document_frequency func - O(|V|):
    since you go through all of the words and find the len ( O(1)) of its documnet dictinary. 
 
D. After the above, we go to the "update_inverted_index_with_tf_idf_and_compute_document_norm" func.
   In this function we're going through all dictionary words keys , twice in cascade --> O(|V|) X 2 = O(|V|)
       For each word- going through all documents in the nested dictionary --> O(|D|) ==> total O(|D||V|)
       
Total Complexity for the BOW model : O(|D||V|)+O(|D||V|)+O(|V|)+ O(|D||V|)= O(|D||V|)
     
       
   
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

  
   We Would have done it by keeping the operations done originally within for loop in the fit fuction without using different   
   functions, avoiding the need the iterate through all of the words in a specific sentence multiple times. 
### END YOUR SOLUTION HERE


## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simmialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.



In [110]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        self.sentence_preprocesser = preprocess_sentence  
        self.vocab = set(tf_idf.unigram_count.keys())
        self.n_docs = tf_idf.n_docs
        self.inverted_index = tf_idf.inverted_index
        self.word_document_frequency = tf_idf.word_document_frequency
        self.doc_norms = tf_idf.doc_norms
    def rank(self,query : Dict[str,int],documents: Dict[str,Counter],metric: str ) -> Dict[int, float]:
        result = {} # key: DocID , value : float , similarity to query
        query_len = np.sum(np.array(list(query.values())))
#given a query and relevant documents calculate the similarity (cosine or inner product simmialrity) 
#between each document and the query.
#Make sure to transform the query word counts to tf idf as well.
        ### YOUR CODE HERE
        #TF IDF ON query:
        for word in query.keys():
            query[word]=((query[word]/query_len)*np.log10(self.n_docs/self.word_document_frequency[word])) 
        d_norm={}
        q_norm = np.sum(np.array(list(query.values()))**2)
        for word in documents.keys():
            #for word in self.inverted_index.keys():
            for doc in documents[word]:
                if doc not in result: result[doc] = 0 
                if word in query: result[doc] += documents[word][doc]*query[word]
                if doc not in d_norm: d_norm[doc] = 0
                d_norm[doc] += documents[word][doc]**2
         ### END YOUR CODE
        if metric == 'cosine':
        ### YOUR CODE HERE
            for doc in result.keys():
                result[doc] = result[doc]/(np.sqrt(q_norm*d_norm[doc]))
        ### END YOUR CODE
        return result
           
    def sort_and_retrieve_k_best(self, scores: Dict[str, float],k :int):
        #returns the top k documents.
        return list(dict(sorted(scores.items(), key=lambda item: item[1],reverse=True)[:k]).keys())
    
    def reduce_query_to_counts(self, query : List)->  Counter:
        #given a list of words returns a counter object with words as keys and counts as values.
        return Counter(query)
        
    def get_top_k_documents(self,query : str, metric: str , k = 5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [111]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [112]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[247101, 247085, 155816, 174957, 313127]
[66192, 61393, 128160, 309297, 318295]


In [113]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[cosine_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
Yeah, I want to dedicate this song to Jonathan Myers,
Morris Peterson, Gemini Smith, and Matthew Hingle.
What's on yo mind?
[Shoestring]
No demonstration on this nation, as a murderfest
Got us locked in the jailcell, the others they was put to rest
I had no teacher, it was like my pops had passed away
Bought me a sweet and snapped his fingers, he was gone away
My house is hell, I used bail down there on ?Acre? street
Whole hood on ABC, pops in penetentiary
Caught in the system, he's a victim of his shorty's past
His son's a killa on the for reala, you best to watch yo stash
What's on my mind, is my brotha's name Rodney King?
Coulda been Shoestring, instead the devil chose Malice Green
Can't go to sleep, not too deep cause I be hearin shots,
Down on my block bodies drop, it'll never stop
The ghetto drama for yo mama is a wicked sin
God save her soul, don't wanna say it but my mom's a fiend
Stand in the rain, can't take the pain

# 1.3 term statistics:
Use "tf_idf" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have (depend on the unique words we have)? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [114]:
# 1. 
### YOUR SOLUTION HERE
print('1.\nUnique words: {}'.format(f'{len(dr.vocab):,}'))
### END YOUR SOLUTION 
print('\nThere are {} unique words.'.format(f'{len(dr.vocab):,}'))


# 2.
### YOUR SOLUTION HERE
print('\n2.\nPotential word bigrams: {}'.format(f'{len(dr.vocab)**2*2:,}'))
print('Actual word bigrams: {}'.format(f'{len(tf_idf.bigram_count.keys()):,}'))
### END YOUR SOLUTION
print("""\nWe dont have all the combinations in the corpus.
The reason is because not all of the combinations are make sense and even out of those that make sense, 
not of all of them are needed in the content of this corpus.""")


# 3.
### YOUR SOLUTION HERE
print('\n3.\nSize of input file is: {} bytes'.format(f'{INPUT_FILE_PATH.stat().st_size:,}'))
print('Size of output file is: {} bytes'.format(f'{BOW_PATH.stat().st_size:,}'))
### END YOUR SOLUTION
print("""\nThe explanation of the diffrence is that the data in these two files is stored differently.
In the input file we are holding every occurrence of a word explicitly.
In the output file we are holding each word just once with list of the relevant documents for this word, with the ranks. 
""")

1.
Unique words: 387,630

There are 387,630 unique words.

2.
Potential word bigrams: 300,514,033,800
Actual word bigrams: 6,288,505

We dont have all the combinations in the corpus.
The reason is because not all of the combinations are make sense and even out of those that make sense, 
not of all of them are needed in the content of this corpus.

3.
Size of input file is: 324,632,382 bytes
Size of output file is: 197,574,542 bytes

The explanation of the diffrence is that the data in these two files is stored differently.
In the input file we are holding every occurrence of a word explicitly.
In the output file we are holding each word just once with list of the relevant documents for this word, with the ranks. 



## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$sim(v,w) := prior \cdot likelihood = p(w) \cdot P(v|w) $$ 
$$P(v|w) := JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [115]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
def get_trigrams(word):
    for ngram in nltk.ngrams(word, 3):
        yield "".join(list(ngram))
        
""" 
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

' \nfor example - get_bigrams is a generator, which is an object we can loop on:\nfor ngram in get_bigrams(word):\n    DO SOMETHING\n'

In [116]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        self.unigram_counts = unigram_counts
        self.ngram_index = {}
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
#In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus 
#to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus. 
#Then, we will implement a function that computes this similarity.  
        for word in self.unigram_counts:
            for gram in self.get_n_grams(word):
                if gram not in self.ngram_index: self.ngram_index[gram] = []
                self.ngram_index[gram].append(word)
             
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        ### YOUR CODE HERE
        result = {} 
        x = set(self.get_n_grams(word))
        relavant_words = [v for k, v in self.ngram_index.items() if k in x] #this is list of lists
        relavant_words = [j for i in relavant_words for j in i] #merge list of lists into one big list
        for w in set(relavant_words):
            y = set(self.get_n_grams(w))
            jaccard_index = len(x.intersection(y))/len(x.union(y))
            #result[w] = jaccard_index
            result[w] = jaccard_index*self.unigram_counts[w]##Or's addition of *p(w)
        return list(sorted(result.items(), key=lambda item: item[1],reverse=True)[:k])

class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)
        
        
class TrigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_trigrams)
        

In [117]:
out_of_vocab_word = 'supercalifragilisticexpialidocious'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

[('like', 9950.375),
 ('caus', 9251.354838709678),
 ('life', 7831.677419354838),
 ('still', 7501.451612903225),
 ('time', 6545.625)]

In [118]:
trigram_spelling_corrector = TrigramSpellingCorrector(tf_idf.unigram_count)
trigram_spelling_corrector.build_index()
trigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

[('life', 3793.46875),
 ('still', 2348.939393939394),
 ('call', 2177.375),
 ('listen', 1374.5454545454545),
 ('hous', 565.34375)]

## 1.5 Language model
Calculate the log likelihood of a sentence. Once with a bigram markovian langauge model, and once with a trigram model.
for example - the likelihood of the senetence "spiderman spiderman does whatever a spider can" for the bigram model is: 
$$p(spiderman)\cdot p(spiderman|spiderman) \cdot  p(does|spiderman) \cdot p(whatever|does) \cdot  p(a|whatever) \cdot  p(spider|a) \cdot p(can|spider)$$

And for the trigram model:
$$p(spiderman,spiderman)\cdot p(does|spiderman,spiderman) \cdot  p(whatever|spiderman,does) \cdot p(a|does,whatever) \cdot  p(spider|whatever,a) \cdot  p(can|a, spider)$$

Since we do not want a zero probability sentence use Laplace smoothing, as you have seen in the lecture, or here https://en.wikipedia.org/wiki/Additive_smoothing

In [119]:
# for the probability smoothing
NUMERATOR_SMOOTHING = 1 # alpha in https://en.wikipedia.org/wiki/Additive_smoothing
DENOMINATOR_SMOOTHING = 10**4 # d in https://en.wikipedia.org/wiki/Additive_smoothing
def sentence_log_probabilty(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str):
    bigram_log_likelilhood, trigram_log_likelilhood = 0, 0
    words_in_sentence = sentence.split()
    n_words = len(words_in_sentence)
    for i, word in  enumerate(words_in_sentence):
        ### YOUR CODE HERE
        if i==0:
            words_counter = np.sum(np.array(list(unigrams.values())))
            #bigram_log_likelilhood=1
            #bigram_log_likelilhood *= ((unigrams[word]+NUMERATOR_SMOOTHING)/(words_counter+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)))
            bigram_log_likelilhood += np.log10(((unigrams[word]+NUMERATOR_SMOOTHING)/(words_counter+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))))
        if i==1:
            words_counter = np.sum(np.array(list(unigrams.values())))
            #bigram_log_likelilhood *= (bigrams[(words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/((unigrams[words_in_sentence[i-1]])+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
            bigram_log_likelilhood += np.log10((bigrams[(words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/((unigrams[words_in_sentence[i-1]])+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)))
            #trigram_log_likelilhood=1
            #trigram_log_likelilhood *= (bigrams[(words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/(words_counter+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
            trigram_log_likelilhood += np.log10((bigrams[(words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/(words_counter+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)))

        if i>=2:
            #bigram_log_likelilhood *= (bigrams[(words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/((unigrams[words_in_sentence[i-1]])+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
            bigram_log_likelilhood += np.log10((bigrams[(words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/((unigrams[words_in_sentence[i-1]])+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)))
            #trigram_log_likelilhood *= (trigrams[(words_in_sentence[i-2],words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/((bigrams[(words_in_sentence[i-1],word)])+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING))
            trigram_log_likelilhood += np.log10((trigrams[(words_in_sentence[i-2],words_in_sentence[i-1],word)]+NUMERATOR_SMOOTHING)/((bigrams[(words_in_sentence[i-1],word)])+(NUMERATOR_SMOOTHING*DENOMINATOR_SMOOTHING)))
        
        ### END YOUR CODE
    return (bigram_log_likelilhood,trigram_log_likelilhood)
    
sentence = "spider man spider man does whatever a spider can"
bigram_log_likelilhood, trigram_log_likelilhood = sentence_log_probabilty(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, sentence)
print(F"Bigram log likelihood is {bigram_log_likelilhood}")
print(F"Trigram log likelihood is {trigram_log_likelilhood}")

Bigram log likelihood is -35.159414861051204
Trigram log likelihood is -32.68473273073879


## 1.51 Language model: B
For each model what is the next word prediciton for the sentnence "like big"?

In [120]:
### YOUR CODE HERE
words_counter = tf_idf.unigram_count.keys()
max_big_bigram=0
max_likebig_trigram=0
for i, word in enumerate(words_counter):
        if tf_idf.bigram_count[("big",word)]>max_big_bigram:
            max_big_bigram=tf_idf.bigram_count[("big",word)]
            pred_word_bigram=word
        if tf_idf.trigram_count[("like","big",word)]>max_likebig_trigram:
            max_likebig_trigram=tf_idf.trigram_count[("like","big",word)]
            pred_word_trigram=word
                           
print("For the bigram model the next word prediction is: %s" %pred_word_bigram)
print("For the trigram model the next word prediction is: %s" %pred_word_trigram)
        

### END YOUR CODE

For the bigram model the next word prediction is: big
For the trigram model the next word prediction is: deal
